In [1]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import pandas as pd

df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


**Basic Text Search**

In [3]:
# keyword filtering:
df[df.course == 'data-engineering-zoomcamp'].head(3)

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."


In [4]:
# vectorization: Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
docs_example = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs_example)

names = vectorizer.get_feature_names_out()  # all different terms
bow_matrix = X.toarray()

print("Vocabulary:", names)
print("BoW Matrix:\n", bow_matrix)

Vocabulary: ['15th' '2024' 'after' 'and' 'before' 'cloud' 'course' 'date' 'for'
 'github' 'google' 'homeworks' 'jan' 'listed' 'not' 'on' 'participation'
 'prerequisites' 'python' 'registration' 'required' 'setup' 'start'
 'starts' 'submit']
BoW Matrix:
 [[1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0]]


In [7]:
# bag of words -> we ignore the order of words, just focus on the words themselves.
df_docs = pd.DataFrame(bow_matrix, columns=names).T
df_docs  # 1: appears in the document; 0: does not appear

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
after,0,0,1,0,0
and,0,0,0,0,1
before,0,0,0,0,1
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
for,0,0,0,1,0
github,0,1,0,0,0


In [8]:
# stop words: Words that are not super relevant as they are very frequent, provide little semantic value 
#             for search tasks and can mess up the vectors if they are not filtered out.
#             For example: and, the, on...

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(docs_example)

names = vectorizer.get_feature_names_out()  # all different terms
bow_matrix = X.toarray()

print("Vocabulary:", names)
print("BoW Matrix:\n", bow_matrix)

Vocabulary: ['15th' '2024' 'cloud' 'course' 'date' 'github' 'google' 'homeworks' 'jan'
 'listed' 'participation' 'prerequisites' 'python' 'registration'
 'required' 'setup' 'start' 'starts' 'submit']
BoW Matrix:
 [[1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0]]


In [9]:
df_docs = pd.DataFrame(bow_matrix, columns=names).T
df_docs  # 1: appears in the document; 0: does not appear

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


In [10]:
# using all documents
vectorizer = CountVectorizer(stop_words='english', min_df=5)
X = vectorizer.fit_transform(df.text)

names = vectorizer.get_feature_names_out()  # all different terms
bow_matrix = X.toarray()

print("Vocabulary:", names)
print("BoW Matrix:\n", bow_matrix)

Vocabulary: ['01' '02' '03' ... 'youtube' 'zip' 'zoomcamp']
BoW Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [11]:
df_docs = pd.DataFrame(bow_matrix, columns=names).T
df_docs  # 1: appears in the document; 0: does not appear

,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0,0,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
yml,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
youtube,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# TF-IDF (Term frequency-Inverse Document Frequency)
# Statistical measure to evaluate how important a word is. More frequent words in a document, but less 
# frequent in the corpus, are more important. It's a way to give more importance to less frequent terms in the corpus.

from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4
15th,0.46,0.00,0.0,0.00,0.00
2024,0.46,0.00,0.0,0.00,0.00
cloud,0.00,0.00,0.0,0.00,0.46
course,0.37,0.00,0.0,0.00,0.37
date,0.00,0.00,0.5,0.00,0.00
github,0.00,0.58,0.0,0.00,0.00
google,0.00,0.00,0.0,0.00,0.46
homeworks,0.00,0.00,0.5,0.00,0.00
jan,0.46,0.00,0.0,0.00,0.00
listed,0.00,0.58,0.0,0.00,0.00


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(stop_words='english', min_df=5)
X = cv.fit_transform(df.text)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
02,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
03,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
04,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
05,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0.0,0.00,0.28,0.0,0.0,0.00,0.21,0.2,0.15,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
yml,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.11,0.0,0.0,0.0,0.00
youtube,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.15,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
zip,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [14]:
# Make a query: We represent the query in the same vector space -i.e. using the same vectorizer.
query = "Do I need to know python to sign up for the January course?"

q = cv.transform([query])
q.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1333))

In [15]:
query_dict = dict(zip(names, q.toarray()[0]))
doc_dict = dict(zip(names, X.toarray()[5]))

In [16]:
# The more words in common - the better the matching score. Let's calculate it:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

print(f'Query: {query}')
print(f'Document: {df.text[5]}')

(df_qd['query'] * df_qd['doc']).sum()  # how similar the query is for document number 1 (using dot product)

Query: Do I need to know python to sign up for the January course?
Document: There are 3 Zoom Camps in a year, as of 2024. However, they are for separate courses:
Data-Engineering (Jan - Apr)
MLOps (May - Aug)
Machine Learning (Sep - Jan)
There's only one Data-Engineering Zoomcamp “live” cohort per year, for the certification. Same as for the other Zoomcamps.
They follow pretty much the same schedule for each cohort per zoomcamp. For Data-Engineering it is (generally) from Jan-Apr of the year. If you’re not interested in the Certificate, you can take any zoom camps at any time, at your own pace, out of sync with any “live” cohort.


np.float64(0.0)

In [17]:
import numpy as np

scores = X.dot(q.T).toarray().ravel()
top5_indices = np.argsort(scores)[::-1][:5]

for idx in top5_indices:
    print(f"Doc {idx} - Score: {scores[idx]:.4f}")
    print(df.text[idx])
    print("-"*100)

Doc 445 - Score: 0.3673
Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)
Introduction to Python – Machine Learning Bookcamp
You can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.
(Mélanie Fouesnard)
----------------------------------------------------------------------------------------------------
Doc 577 - Score: 0.2524
You can find the intercept between these two curves using numpy diff (https://numpy.org/doc/stable/reference/generated/numpy.diff.html ) and sign (https://numpy.org/doc/stable/reference/genera

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(X, q)

cs[:10]

array([[0.19464486],
       [0.        ],
       [0.        ],
       [0.06011641],
       [0.04932915],
       [0.        ],
       [0.        ],
       [0.13477565],
       [0.        ],
       [0.        ]])

In [19]:
# Tfidf Vectorizer for field

fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

# Text field
print(transformers['text'].get_feature_names_out())
print(matrices['text'][0])

['001' '01' '02' ... 'zones' 'zoom' 'zoomcamp']
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34 stored elements and shape (1, 2118)>
  Coords	Values
  (0, 1497)	0.15309145549916028
  (0, 571)	0.12832805481697965
  (0, 279)	0.15309145549916028
  (0, 790)	0.1633284003128425
  (0, 183)	0.12832805481697965
  (0, 1882)	0.1633284003128425
  (0, 1517)	0.10519408479610691
  (0, 673)	0.15769460303990118
  (0, 489)	0.14285451068547808
  (0, 892)	0.1633284003128425
  (0, 440)	0.46862657292357235
  (0, 1003)	0.14582822756491107
  (0, 31)	0.12995748547828756
  (0, 1803)	0.10222036791667394
  (0, 1292)	0.14285451068547808
  (0, 893)	0.12995748547828756
  (0, 1092)	0.14019443029196976
  (0, 1494)	0.12028792212496518
  (0, 831)	0.09070710886127974
  (0, 2082)	0.11419920421207885
  (0, 529)	0.12398928050275179
  (0, 1570)	0.30618291099832057
  (0, 1807)	0.14582822756491107
  (0, 2003)	0.061474263046700436
  (0, 1083)	0.0962521629411382
  (0, 1010)	0.2803888605839395
  (0, 1885)	0.16332

In [20]:
query = "I just signed up. Is it too late to join the course?"

q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

idx = np.argsort(-score)[:10]
df.iloc[idx].text

0      The purpose of this document is to capture fre...
449    Yes, you can. You won’t be able to submit some...
440    The process is automated now, so you should re...
15     No, late submissions are not allowed. But if t...
22     It's up to you which platform and environment ...
448    Here’s how you join a in Slack: https://slack....
797    Depends on whether the form will still be open...
764    If you have submitted two projects (and peer-r...
27     You can do most of the course without a cloud....
452    Welcome to the course! Go to the course page (...
Name: text, dtype: object

In [21]:
# Add boost
boost = {'question': 3.0}
score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s

# creating filters
filters = {
    'course': 'data-engineering-zoomcamp'
}

for field, value in filters.items():
    mask = (df[field] == value).values
    score = score * mask

In [22]:
idx = np.argsort(-score)[:10]
results = df.iloc[idx]
results.to_dict(orient='records')

[{'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - Which playlist on YouTube should I refer to?',
  'text': 'All the main videos are stored in the Main “DATA ENGINEERING” playlist (no year specified). The Github repository has also been updated to show each video with a thumbnail, that would bring you directly to the same playlist below.\nBelow is the MAIN PLAYLIST’. And then you refer to the

**Putting it all together**

In [23]:
class TextSearch:
    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.transformers = {}
        self.matrices = {}

    def fit(self, records, vectorizer_params:dict = {}):
        self.dataframe = pd.DataFrame(records)

        for field in self.text_fields:
            vectorizer = TfidfVectorizer(**vectorizer_params)
            X = vectorizer.fit_transform(self.dataframe[field])

            self.transformers[field] = vectorizer
            self.matrices[field] = X

    def search(self, query:str, k:int = 5, boost:dict = {}, filters:dict = {}, return_scores=False):
        if not hasattr(self, "dataframe"):
            raise RuntimeError("You must call .fit(records) before searching.")

        score = np.zeros(len(self.dataframe))

        for field in self.text_fields:
            boost_value = boost.get(field, 1.0)
            query_mtx = self.transformers[field].transform([query])
            similarity = cosine_similarity(self.matrices[field], query_mtx).flatten()
            
            score += boost_value * similarity

        for field, value in filters.items():
            mask = (self.dataframe[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:k]
        results = self.dataframe.iloc[idx]
        results_records = results.to_dict(orient='records')

        if return_scores:
            scoring = score[idx].round(6).tolist()
            for r, s in zip(results_records, scoring):
                r['relevance_score'] = s
        return results_records

In [24]:
index = TextSearch(
    text_fields=['section', 'question', 'text']
)
index.fit(documents)

relevant_docs = index.search(
    query='I just signed up. Is it too late to join the course?',
    k=5,
    boost={'question': 2.0},
    filters={'course': 'data-engineering-zoomcamp'},
    return_scores=True
)

In [25]:
import json

print(json.dumps(relevant_docs, indent=4, ensure_ascii=False))

[
    {
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        "section": "General course-related questions",
        "question": "Course - Can I still join the course after the start date?",
        "course": "data-engineering-zoomcamp",
        "relevance_score": 0.969895
    },
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related qu

**SVD**

In [26]:
X = matrices['text']  # sparse matrix
text_vectorizer = transformers['text']  # vectorizer for text field

In [27]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=16)  # vectors --> embeddings
X_emb = svd.fit_transform(X)

print(f'Old dimensionality: {X.shape}')
print(f'New dimensionality: {X_emb.shape}')

Old dimensionality: (948, 2118)
New dimensionality: (948, 16)


In [28]:
import numpy as np

terms = text_vectorizer.get_feature_names_out()
for i, comp in enumerate(svd.components_):
    terms_in_topic = np.argsort(comp)[-10:]  # top 10 words for this component
    print(f"Tema {i}: {[terms[t] for t in terms_in_topic]}")

Tema 0: ['use', 'https', 'command', 'com', 'python', 'file', 'install', 'run', 'error', 'docker']
Tema 1: ['5432', 'run', 'images', 'command', 'network', 'container', 'build', 'image', 'compose', 'docker']
Tema 2: ['command', 'path', 'environment', 'jupyter', 'conda', 'version', 'pipenv', 'pip', 'python', 'install']
Tema 3: ['datatalksclub', 'blob', 'master', 'pip', 'zoomcamp', 'python', 'install', 'com', 'https', 'github']
Tema 4: ['description', 'image', 'sklearn', 'https', 'added', 'docker', 'scikit', 'learn', 'features', 'model']
Tema 5: ['url', 'files', 'file', 'pd', 'pandas', 'spark', 'parquet', 'data', 'csv', 'df']
Tema 6: ['apache', 'sparksession', 'worker', '11', 'pyspark', 'model', 'version', 'master', 'java', 'spark']
Tema 7: ['jupyter', 'notebook', 'ny_taxi', 'localhost', 'postgres', 'postgresql', 'ssh', 'root', 'port', '5432']
Tema 8: ['model', 'git', 'config', 'file', 'solution', 'ssh', 'added', 'aws', 'problem', 'description']
Tema 9: ['python', 'cohort', 'problem', 'pro

In [29]:
query = 'I just signed up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.04353728, -0.03058995, -0.04425823,  0.01273177,  0.02584644,
       -0.04814667,  0.01030718,  0.03020395, -0.12337402,  0.16903517,
        0.02680219,  0.0771933 , -0.05028467, -0.03613067,  0.05507175,
       -0.01514956])

In [30]:
# calculate similarity
np.dot(X_emb[0], Q_emb[0])

np.float64(0.11487444694720117)

In [31]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:5]
list(df.loc[idx].text)

['If you have submitted two projects (and peer-reviewed at least 3 course-mates’ projects for each submission), you will get the certificate for the course. According to the course coordinator, Alexey Grigorev, only two projects are needed to get the course certificate.\n(optional) David Odimegwu',
 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
 "The purpose of this document is to

**NMF**

In [32]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)

print(f'Old dimensionality: {X.shape}')
print(f'New dimensionality: {X_emb.shape}')

Old dimensionality: (948, 2118)
New dimensionality: (948, 16)


In [33]:
X_emb[1]

# Each array position represents how much a topic contributes to the document.
# It is a non-negative additive combination (all values are ≥ 0), which means:

    # Topic 11: contributes 0.1756 → the strongest in this document.
    # Subject 5: contributes 0.0463.
    # Topic 7: contributes 0.0121.

# The rest of the topics: contribute nothing (weight 0).

# The document is mainly about data file handling: CSVs, parquet, downloads, etc

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.04510525, 0.        , 0.01274025, 0.        , 0.        ,
       0.        , 0.17412189, 0.        , 0.        , 0.        ,
       0.        ])

In [34]:
H = nmf.components_
terms = text_vectorizer.get_feature_names_out()

for i, topic in enumerate(H):
    top_words = [terms[i] for i in topic.argsort()[-10:][::-1]]
    print(f"Tema {i}: {top_words}")

Tema 0: ['ssh', 'file', 'folder', 'config', 'key', 'path', 'vm', 'user', 'ip', 'git']
Tema 1: ['docker', 'compose', 'image', 'build', 'container', 'run', 'command', 'images', 'error', 'network']
Tema 2: ['install', 'pip', 'pipenv', 'conda', 'command', 'environment', 'lock', 'pipfile', 'windows', 'apt']
Tema 3: ['dbt', 'bigquery', 'project', 'location', 'cloud', 'dataset', 'branch', 'error', 'create', 'created']
Tema 4: ['model', 'features', 'training', 'train', 'validation', 'fit', 'feature', 'regression', 'values', 'categorical']
Tema 5: ['data', 'df', 'csv', 'parquet', 'pandas', 'files', 'file', 'pd', 'table', 'gz']
Tema 6: ['spark', 'master', 'java', 'pyspark', 'worker', 'sparksession', 'apache', 'bin', 'spark_home', '7077']
Tema 7: ['5432', 'root', 'postgresql', 'port', 'postgres', 'ny_taxi', 'var', 'lib', 'pgcli', 'localhost']
Tema 8: ['description', 'problem', 'solution', 'added', 'optional', 'commit', 'pre', 'run', 'error', 'calderin']
Tema 9: ['course', 'yes', 'projects', 'answ

In [35]:
Q = cv.transform([query])
Q_emb = nmf.transform(Q)
Q_emb[0]

# The document focuses on general course information, participation, communication, 
# project delivery and certificates.

array([0.        , 0.00117982, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.17120171,
       0.        , 0.        , 0.        , 0.        , 0.00047833,
       0.        ])

In [36]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:3]
list(df.loc[idx].text)

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.',
 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.']

**BERT**

In [37]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [38]:
# tokenizer - for turning text into vectors
# model - for compressing the text into embeddings

texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]

encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encoded_input

{'input_ids': tensor([[  101,  2748,  1010,  2057,  2097,  2562,  2035,  1996,  4475,  2044,
          1996,  2607, 12321,  1012,   102],
        [  101,  2017,  2064,  3582,  1996,  2607,  2012,  2115,  2219,  6393,
          2044,  2009, 12321,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [40]:
# Compute the embeddings:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [41]:
# Now we need to compress the embeddings:
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings.shape  # for each document we need a matrix

torch.Size([2, 768])

In [42]:
# And convert them to a numpy array
X_emb = sentence_embeddings.numpy()

In [45]:
sentence_embeddings_cpu = sentence_embeddings.cpu()

In [46]:
# Let's now compute it for our texts. We'll do it in batches. First, we define a function for batching:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result


def compute_embeddings(texts, batch_size=8):
    text_batches = make_batches(texts, 8)
    
    all_embeddings = []
    
    for batch in tqdm(text_batches):
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    
        with torch.no_grad():
            outputs = model(**encoded_input)
            hidden_states = outputs.last_hidden_state
            
            batch_embeddings = hidden_states.mean(dim=1)
            batch_embeddings_np = batch_embeddings.cpu().numpy()
            all_embeddings.append(batch_embeddings_np)
    
    final_embeddings = np.vstack(all_embeddings)
    return final_embeddings

In [47]:
from tqdm.auto import tqdm
texts = df['text'].tolist()
text_batches = make_batches(texts, 8)

all_embeddings = []

for batch in tqdm(text_batches):
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoded_input)
        hidden_states = outputs.last_hidden_state
        
        batch_embeddings = hidden_states.mean(dim=1)
        batch_embeddings_np = batch_embeddings.cpu().numpy()
        all_embeddings.append(batch_embeddings_np)

final_embeddings = np.vstack(all_embeddings)

  0%|          | 0/119 [00:00<?, ?it/s]

In [48]:
final_embeddings

array([[-0.00456303, -0.11667512,  0.6274718 , ..., -0.03659191,
         0.10031679,  0.0292713 ],
       [-0.1423361 , -0.1985392 ,  0.28455415, ..., -0.01139053,
        -0.1539977 ,  0.09535079],
       [ 0.19672246, -0.08461305,  0.28200513, ...,  0.11395867,
        -0.06448027, -0.0128261 ],
       ...,
       [-0.2821744 , -0.33324358,  0.29784983, ..., -0.35042733,
         0.03266054,  0.09537254],
       [-0.42807093, -0.39468756,  0.3094198 , ..., -0.05943285,
        -0.12965176,  0.07887058],
       [-0.16892126, -0.25146285,  0.47843292, ..., -0.18535416,
        -0.1610892 ,  0.27272922]], shape=(948, 768), dtype=float32)

In [ ]:
# fields = ['section', 'question', 'text']

for f in fields:
    print(f'computing embeddings for {f}...')
    embeddings[f] = compute_embeddings(df[f].tolist())

In [ ]:
import pickle

with open('embeddings.bin', 'wb') as f_out:
    pickle.dump(embeddings)